## Kruispunt Simuleren In Python 
----------------------------------------------
**Student : Hussin Almoustafa** 

## inleiding 
Waarom verkeersstroom simuleren?
De belangrijkste reden achter het simuleren van verkeer is het genereren van gegevens zonder de echte wereld. In plaats van nieuwe ideeën te testen over hoe verkeerssystemen in de echte wereld te beheren of gegevens te verzamelen met behulp van sensoren, kunt u een op software draaiend model gebruiken om de verkeersstroom te voorspellen. Dit helpt de optimalisatie en gegevensverzameling van verkeerssystemen te versnellen. Simulatie is een veel goedkoper en sneller alternatief voor testen in de echte wereld.

## Modellering
Om verkeerssystemen te analyseren en te optimaliseren, moeten we eerst een verkeerssysteem wiskundig modelleren. Een dergelijk model moet de verkeersstroom realistisch weergeven op basis van invoerparameters (geometrie van het wegennet, voertuigen per minuut, voertuigsnelheid, …).

In dit Simulatie zal ik een microscopisch model gebruiken.


**Een microscopisch** bestuurdersmodel beschrijft het gedrag van een enkele bestuurder/voertuig. Daarom moet het een multi-agent systeem zijn, dat wil zeggen dat elk voertuig op zichzelf werkt met input uit zijn omgeving.
![Dierction of traffic](1.png)


In microscopische modellen is elk voertuig genummerd met een nummer i. Het i-de voertuig volgt het (i-1)-de voertuig. Voor het i-de voertuig zullen we de positie langs de weg met xᵢ aangeven, vᵢ de snelheid en lᵢ de lengte. En dat geldt voor elk voertuig. 

![microscopische](3.png)

Met sᵢ geven we de bumper-tot-bumper afstand aan en Δvᵢ het snelheidsverschil tussen het i-de voertuig en het voertuig ervoor (voertuignummer i-1).



**Voertuiggenerator**

Om voertuigen aan onze simulatie toe te voegen, hebben we twee opties:
* Voeg elk voertuig handmatig toe aan de simulatie door een nieuwe voertuigklasse-instantie te maken en deze toe te voegen aan de lijst met voertuigen.

* Voeg voertuigen toe volgens vooraf gedefinieerde kansen.

Voor de tweede optie moeten we een stochastische voertuiggenerator definiëren.

Een voertuiggenerator wordt gedefinieerd door twee beperkingen:
* Voertuiggeneratiesnelheid (τ): (in voertuigen per minuut) beschrijft hoeveel voertuigen gemiddeld per minuut aan de simulatie moeten worden toegevoegd.

* Voertuigconfiguratielijst (L): Een lijst met tupels die de configuratie en waarschijnlijkheid van voertuigen bevatten.
L = [(p₁, V₁), (p₂, V₂), (p₃, V₃), …]

De voertuiggenerator genereert het voertuig Vᵢ met waarschijnlijkheid pᵢ.

**Verkeerslicht** 

Verkeerslichten worden op hoeken geplaatst en worden gekenmerkt door twee zones:

Vertragingszone: gekenmerkt door een vertragingsafstand en een vertragingsfactor, is een zone waarin voertuigen hun maximale snelheid vertragen met behulp van de vertragingsfactor.
![Vertragingszone](2.png)

Stopzone: gekenmerkt door een stopafstand, is een zone waarin voertuigen stoppen. Dit wordt bereikt met behulp van een dempingskracht door deze dynamische vergelijking:
![Stopzone](4.png)


**Simulatie**

We gaan voor een objectgeoriënteerde aanpak. Elk voertuig en elke weg wordt als een klasse gedefinieerd.
We zullen de volgende __init__-functie herhaaldelijk gebruiken in veel komende lessen. Het stelt de standaardconfiguratie van de huidige klasse in via een functie set_default_config. En verwacht een woordenboek en stelt elke eigenschap in het woordenboek in als een eigenschap voor de huidige klasse-instantie. Op deze manier hoeven we ons geen zorgen te maken over het bijwerken van __init__-functies van verschillende klassen of over toekomstige wijzigingen.

In [1]:
def __init__(self, config={}):
  # Set default configuration
  self.set_default_config()

  # Update configuration
  for attr, val in config.items():
      setattr(self, attr, val)

**voertuigen** 

We zullen Taylorreeks gebruiken om de oplossing van de dynamische vergelijkingen te benaderen.

Het is gebruikelijk een functie te benaderen door een eindig aantal termen van haar taylorreeks te gebruiken. De stelling van Taylor geeft kwantitatieve schattingen van de fout in deze benadering.


Taylorreeksexpansie voor een oneindig differentieerbare functie f is:
![Taylorreeksexpansie](5.png)

In elke iteratie (of frame), na het berekenen van de versnelling met behulp van de IDM-formule, zullen we de positie en snelheid bijwerken met behulp van deze twee vergelijkingen:

![positie en snelheid](6.png)

**Curven** 

In de echte wereld hebben wegen bochten. Hoewel we technisch gezien bochten kunnen maken in deze simulatie door de coördinaten van veel wegen met de hand te schrijven om een bocht te benaderen, kunnen we hetzelfde procedureel doen.


Ik zal hiervoor bézierkromme  gebruiken.
![positie en snelheid](7.png)




**Implentatie**

We zullen een Weg-klasse maken:


In [2]:
from scipy.spatial import distance

class Weg:
    def __init__(self, start, end):
        self.start = start
        self.end = end

        self.init_porperties()

    def init_properties(self):
        self.length = distance.euclidean(self.start, self.end)
        self.angle_sin = (self.end[1]-self.start[1]) / self.length
        self.angle_cos = (self.end[0]-self.start[0]) / self.length

We hebben de lengte van de weg en de cosinus en sinus van de hoek nodig wanneer we deze op het scherm tekenen.

**Simulation** 

En een simulatieles. Ik heb een aantal methoden toegevoegd om wegen aan de simulatie toe te voegen. 

In [3]:

class Simulation:
    def __init__(self, config={}):
        # Standaardconfiguratie instellen
        self.set_default_config()

        # Configuratie bijwerken
        for attr, val in config.items():
            setattr(self, attr, val)

    def set_default_config(self):
        self.t = 0.0            # Tijd 
        self.frame_count = 0    # Frame telling
        self.dt = 1/60          # Simulatie tijdstap
        self.wegen = []         # Array om wegen op te slaan

    def maak_weg(self, start, eind):
        weg = Weg(start, eind)
        self.wegen.append(weg)
        return weg

    def maak_wegen(self, wegen_lijst):
        for weg in wegen_lijst:
            self.maak_weg(*road)

**Voertuigen** 

In [4]:
import numpy as np

class Voertuigen:
    def __init__(self, config={}):
        # Standaardconfiguratie instellen
        self.set_default_config()

        # Configuratie bijwerken
        for attr, val in config.items():
            setattr(self, attr, val)

        # Bereken properties
        self.init_properties()

    def set_default_config(self):    
        self.l = 4
        self.s0 = 4
        self.T = 1
        self.v_max = 16.6
        self.a_max = 1.44
        self.b_max = 4.61

        self.path = []
        self.huidige_wegenindex = 0

        self.x = 0
        self.v = self.v_max
        self.a = 0
        self.gestopt = False

    def init_properties(self):
        self.sqrt_ab = 2*np.sqrt(self.a_max*self.b_max)
        self._v_max = self.v_max

    def update(self, lead, dt):
        """Aangezien dit slechts een benadering is, kan de snelheid soms negatief
        worden (maar het model laat dat niet toe). Een instabiliteit ontstaat 
        wanneer de snelheid negatief is,
        en de positie en snelheid divergeren naar negatief oneindig."""
        if self.v + self.a*dt < 0:
            self.x -= 1/2*self.v*self.v/self.a
            self.v = 0
        else:
            self.v += self.a*dt
            self.x += self.v*dt + self.a*dt*dt/2
            
        """ Om de IDM-versnelling te berekenen, zullen we het lead voertuigaanduiden 
            als lead en de interactieterm berekenen
            (aangeduid met alpha) wanneer lead is Geen None."""
        # Acceleratie updaten
        alpha = 0
        if lead:
            delta_x = lead.x - self.x - lead.l
            delta_v = self.v - lead.v

            alpha = (self.s0 + max(0, self.T*self.v + delta_v*self.v/self.sqrt_ab)) / delta_x

        self.a = self.a_max * (1-(self.v/self.v_max)**4 - alpha**2)
        
        """ Als het voertuig stilstaat (bijvoorbeeld bij een verkeerslicht),
            gebruiken we de dempingsvergelijking:"""

        if self.gestopt:
            self.a = -self.b_max*self.v/self.v_max
        
    def stop(self):
        self.gestopt = True

    def ontstoppen(self):
        self.gestopt = False

    def langzaam(self, v):
        self.v_max = v

    def niet_vertragen(self):
        self.v_max = self._v_max


**Voertuiggeneratoren** 

VehicleGenerator heeft een reeks tupels met (odds, voertuig) .
Het eerste element van de tupel is het gewicht (niet de waarschijnlijkheid) van het genereren van het voertuig in dezelfde tupel. Ik heb gewichten gebruikt omdat ze gemakkelijker zijn om mee te werken, omdat we gewoon gehele getallen kunnen gebruiken.
Als we bijvoorbeeld 3 voertuigen hebben met gewichten 1 , 3 , 2 . Dit komt overeen met 1/6 , 3/6 , 2/6 met 6=1+3+2.


In [5]:
from numpy.random import randint

class Voertuiggenerator:
    def __init__(self, sim, config={}):
        self.sim = sim

        # Standaardconfiguraties instellen
        self.set_default_config()

        # update configuraties
        for attr, val in config.items():
            setattr(self, attr, val)

        # bereken properties
        self.init_properties()

    def set_default_config(self):
        """Standaardconfiguratie instellen"""
        self.voertuig_tarief = 20
        self.voertuigen = [
            (1, {})
        ]
        self.laatst_toegevoegde_tijd = 0

    def init_properties(self):
        self.aankomend_voertuig = self.voertuig_genereren()

    def voertuig_genereren(self):
        """Retourneert een willekeurig voertuig van zelf.voertuigen met willekeurige verhoudingen"""
        totaal = sum(pair[0] for pair in self.voertuigen)
        r = randint(1, totaal+1)
        for (weight, config) in self.voertuigen:
            r -= weight
            if r <= 0:
                return Voertuigen(config)

    def update(self):
        """Voertuigen toevoegen"""
        if self.sim.t - self.laatst_toegevoegde_tijd >= 60 / self.voertuig_tarief:
            # Als de tijd is verstreken nadat het laatst toegevoegde voertuig is
            # langer dan voertuigperiode; een voertuig genereren
            weg = self.sim.wegen[self.aankomend_voertuig.path[0]]
            if len(weg.voertuigen) == 0\
               or weg.voertuigen[-1].x > self.aankomend_voertuig.s0 + self.aankomend_voertuig.l:
                # Als er ruimte is voor het gegenereerde voertuig; voeg het toe
                self.aankomend_voertuig.time_added = self.sim.t
                weg.voertuigen.append(self.aankomend_voertuig)
                # Reset laatst toegevoegde tijd en aankomend voertuig
                self.laatst_toegevoegde_tijd = self.sim.t
            self.aankomend_voertuig = self.voertuig_genereren()



**Verkeerslicht**


self.cycle is een array van tupels die de toestanden bevatten (True voor groen en False voor rood) voor elke weg in self.wegen .
In de standaardconfiguratie betekent (False, True) dat de eerste set weg rood is en de tweede groen. (Waar, niet waar) is het tegenovergestelde.
Deze aanpak wordt gebruikt omdat het gemakkelijk schaalbaar is. We creëren verkeerslichten die meer dan 2 wegen omvatten, verkeerslichten met aparte signalen voor afslagen naar rechts en naar links, of zelfs voor gesynchroniseerde verkeerslichten over meerdere kruispunten.
De update-functie van een verkeerslicht zou aanpasbaar moeten zijn. Het standaardgedrag is symmetrisch fietsen met een vaste tijd.

In [6]:
class Verkeerslicht:
    def __init__(self, wegen, config={}):
        # Wegen initialiseren
        self.wegen = wegen
        # Standaard configuratie instellen
        self.set_default_config()
        # Update configuratie
        for attr, val in config.items():
            setattr(self, attr, val)
        # bereken properties
        self.init_properties()

    def set_default_config(self):
        self.cycle = [(True, False), (False, True)]
        self.langzame_afstand = 45
        self.langzame_factor = 0.3
        self.stop_afstand = 10
        self.satus = 0

        self.huidige_cyclusindex = 0

        self.last_t = 0

    def init_properties(self):
        for i in range(len(self.wegen)):
            for weg in self.wegen[i]:
                weg.verkeerslicht_instellen(self, i)

    @property
    def huidige_cyclus(self):
        return self.cycle[self.huidige_cyclusindex]
    
    def update(self, sim):
        cycle_lengte = 30
        k = (sim.t // cycle_lengte) % 2
        self.huidige_cyclusindex = int(k)



**Bocht functies** 

bevat functies die helpen bij het maken van bochten en ernaar verwijzen door hun wegindexen.

In [7]:

def bocht_punten(start, eind, control, resolution=5):
    
    # Als curve een rechte lijn is
    if (start[0] - eind[0])*(start[1] - eind[1]) == 0:\
        
        return [start, eind]

    #Zo niet, retourneer dan een curve
    path = []

    for i in range(resolution+1):
        t = i/resolution
        x = (1-t)**2 * start[0] + 2*(1-t)*t * control[0] + t**2 *eind[0]
        y = (1-t)**2 * start[1] + 2*(1-t)*t * control[1] + t**2 *eind[1]
        path.append((x, y))

    return path

def bocht_weg(start, eind, control, resolution=15):
    punten = bocht_punten(start, eind, control, resolution=resolution)
    return [(punten[i-1], punten[i]) for i in range(1, len(punten))]

SLA_LINKSAF = 0
SLA_RECHTSAF = 1
def weg_afslaan(start, eind, draai_richting, resolution=15):
    # Krijg controlepunt
    x = min(start[0], eind[0])
    y = min(start[1], eind[1])

    if draai_richting == SLA_LINKSAF:
        control = (x - y + start[1], y - x + eind[0])
    else:
        control = (
        x - y + eind[1],
        y - x + start[0])
        
    return bocht_weg(start, eind, control, resolution=resolution)

**Conclusie**

Hoewel we de Class Simulation kunnen aanpassen om gegevens over onze simulatie op te slaan die we later kunnen gebruiken, zou het beter zijn als het proces voor het verzamelen van gegevens meer gestroomlijnd was.
Aan deze simulatie ontbreekt nog veel. De uitvoering van bochten is slecht en inefficiënt en veroorzaakt problemen met interacties tussen voertuigen en verkeerslichten.